In [3]:
import numpy as np
import numpy.linalg as la
import numpy.random as rn
import matplotlib.pyplot as plt
import scipy.io as io
from scipy.sparse import csr_matrix, rand
from scipy import sparse

In [4]:
y_train = np.loadtxt(open("news/y_news_train.csv", "rb"), delimiter=",")
y_test = np.loadtxt(open("news/y_news_test.csv", "rb"), delimiter=",")

In [11]:
X_train = io.mmread("news/X_news_train.mtx")
X_test = io.mmread("news/X_news_test.mtx")
C = 20

In [11]:
print(X_train.shape)

(12564, 101631)


In [8]:
def objective(X, y, beta, C):
    N = X.shape[0]
    sizeX = X.shape[1]
    #obj3 = mu*la.norm(beta)
    
    obj = 0
    tempVec = np.zeros(C)
    for i in range(X.shape[0]):
        x = X[i,:]
        obj1 = np.dot(beta[int(y[i]*X.shape[1]):int((y[i]*X.shape[1]+X.shape[1]))],x)
        for j in range(C):
            tempVec[j] = np.exp(-np.dot(beta[j*sizeX:(j*sizeX+sizeX)], x))
        obj2 = np.log(np.sum(tempVec))
        
        obj += obj1 + obj2
    
    obj = 1/N*(obj) #+ obj3
    return obj

In [24]:
def gradient1(x, y, beta, C):
    sizeBeta = beta.shape[0]
    sizeX = sizeBeta/C
    grad1 = np.zeros(sizeBeta)
    grad1[int(y*sizeX):int(y*sizeX + sizeX)] = x
    
    
    tempVec = np.zeros(C)
    tempVec2 = np.zeros(sizeBeta)
    
    for j in range(C):
        betatemp = beta[int(j*sizeX):int(j*sizeX+sizeX)]
        tempVec[j] = np.exp(-np.dot(betatemp, x))
        
        tempVec2[int(j*sizeX):int(j*sizeX+sizeX)] = -x*tempVec[j]
    
    grad2 = tempVec2/np.sum(tempVec)
    
    return (grad1 + grad2)

In [6]:
def minibatch(X, Y, beta, C, N, t):
    eta = 1e-3/np.sqrt(t + 1)
    
    gradient = 0
    
    randset = rn.randint(X.shape[0], N)
    
    for i in range(N):
        x = X[randset[i],:]
        y = Y[randset[i]]
        
        gradient += gradient1(x, y, beta, C)
    
    beta = beta - eta*gradient
    
    return beta

In [7]:
def stochgradient(x, y, beta, C, mu, N, t):
    eta = 1e-3/np.sqrt(t+1)
    
    beta = beta - eta*gradient1(x, y, beta, C, N, mu)
    
    return beta

In [9]:
def descent(update, X, Y, X_test, Y_test, C, mu):
    train = []
    test = []
    
    beta = np.zeros(X.shape[1]*C)
    
    T = 10
    
    for t in range(T):
        beta = update(X, Y, beta, C, mu, t)
        
        if(t % 1 == 0) or (t == T - 1):
            train.append(objective(X, Y, beta, C))
            test.append(objective(X_test, Y_test, beta, C))
            
            assert not np.isnan(train[-1])
        
    return beta, train, test

In [10]:
def descent2(update, X, Y, X_test, Y_test, C, mu):
    train = []
    test = []
    
    beta = np.zeros(X.shape[1]*C)
    N = X.shape[1]
    
    T = 10
    
    for t in range(T):
        i = rn.randint(X_train.shape[0])
        x = X[i,:]
        y = Y[i]
        beta = update(x, y, beta, C, mu, N, t)
        
        if(t % 1 == 0) or (t == T - 1):
            train.append(objective(X, Y, beta, C))
            test.append(objective(X_test, Y_test, beta, C))
            
            assert not np.isnan(train[-1])
            
    return beta, train, test

In [ ]:
beta_mb, train_mb, test_mb = descent(minibatch, X_train, y_train, X_test, y_test, C, 1)

In [ ]:
beta_sg, train_sg, test_sg = descent2(stochgradient, X_train, y_train, X_test, y_test, C, 1)

Code is correct. But runtime is too long. 